In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('HMXPC13_DI_v2_5-14-14.csv')

In [22]:
data.head(10)

,course_id,userid_DI,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts
0,HarvardX/CB22x/2013_Spring,MHxPC130442623,0,0,0,United States,NaN,NaN,NaN,0,2012-12-19,2013-11-17,0.0,9.0,0.0,0.0,0
1,HarvardX/CS50x/2012,MHxPC130442623,1,0,0,United States,NaN,NaN,NaN,0,2012-10-15,NaN,0.0,9.0,0.0,1.0,0
2,HarvardX/CB22x/2013_Spring,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2013-02-08,2013-11-17,0.0,16.0,0.0,0.0,0
3,HarvardX/CS50x/2012,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2012-09-17,NaN,0.0,16.0,0.0,0.0,0
4,HarvardX/ER22x/2013_Spring,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2012-12-19,NaN,0.0,16.0,0.0,0.0,0
5,HarvardX/PH207x/2012_Fall,MHxPC130275857,1,1,0,United States,NaN,NaN,NaN,0,2012-09-17,2013-05-23,502.0,16.0,50.0,12.0,0
6,HarvardX/PH278x/2013_Spring,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2013-02-08,NaN,0.0,16.0,0.0,0.0,0
7,HarvardX/CB22x/2013_Spring,MHxPC130539455,1,0,0,France,NaN,NaN,NaN,0,2013-01-01,2013-05-14,42.0,6.0,0.0,3.0,0
8,HarvardX/CB22x/2013_Spring,MHxPC130088379,1,0,0,United States,NaN,NaN,NaN,0,2013-02-18,2013-03-17,70.0,3.0,0.0,3.0,0
9,HarvardX/CS50x/2012,MHxPC130088379,1,0,0,United States,NaN,NaN,NaN,0,2012-10-20,NaN,0.0,12.0,0.0,3.0,0


In [4]:
data.columns

Index(['course_id', 'userid_DI', 'registered', 'viewed', 'explored',
       'certified', 'final_cc_cname_DI', 'LoE_DI', 'YoB', 'gender', 'grade',
       'start_time_DI', 'last_event_DI', 'nevents', 'ndays_act', 'nplay_video',
       'nchapters', 'nforum_posts', 'roles', 'incomplete_flag'],
      dtype='object')

In [5]:
data.shape

(641138, 20)

In [7]:
data = data.drop(['roles', 'incomplete_flag' , 'registered'], axis=1) #We are dropping 3 columns: registered becasue it has only
#value 1 and nothing else; incoplete flag and roles because they have mostly Null values and are only used for administration

In [9]:
data['nevents'].value_counts().sort_index() #checking if column "nevents" has 0 value.If it has no 0 values we can treat NaN as 0

1.0         63565
2.0         34329
3.0         17669
4.0         12217
5.0          9850
6.0          8480
7.0          7259
8.0          6648
9.0          6076
10.0         5621
11.0         5197
12.0         4870
13.0         4476
14.0         4222
15.0         3891
16.0         3808
17.0         3545
18.0         3349
19.0         3144
20.0         3064
21.0         2956
22.0         2795
23.0         2578
24.0         2621
25.0         2420
26.0         2391
27.0         2216
28.0         2170
29.0         2122
30.0         2079
            ...  
30417.0         1
30659.0         1
30810.0         1
30849.0         1
31396.0         1
31416.0         1
31761.0         1
31868.0         1
31900.0         1
31904.0         1
31943.0         1
32296.0         1
32498.0         1
32528.0         1
32812.0         1
33650.0         1
33661.0         1
33820.0         1
36244.0         1
37440.0         1
38065.0         1
41423.0         1
41895.0         1
43880.0         1
45201.0   

In [11]:
data['ndays_act'].value_counts().sort_index() #checking if column "ndays_act" has 0 value.

1.0      209941
2.0       80625
3.0       43081
4.0       26813
5.0       18552
6.0       13239
7.0       10281
8.0        8075
9.0        6510
10.0       5324
11.0       4415
12.0       3815
13.0       3323
14.0       2794
15.0       2542
16.0       2146
17.0       2002
18.0       1852
19.0       1564
20.0       1497
21.0       1339
22.0       1309
23.0       1207
24.0       1093
25.0       1027
26.0        947
27.0        887
28.0        891
29.0        838
30.0        806
          ...  
130.0         8
131.0         4
132.0         4
133.0         4
134.0         3
135.0         2
136.0         5
137.0         1
138.0         4
139.0         1
140.0         1
141.0         4
142.0         2
143.0         4
144.0         1
147.0         3
148.0         1
149.0         1
150.0         1
151.0         1
153.0         1
155.0         3
157.0         1
158.0         1
159.0         2
160.0         1
162.0         1
165.0         1
176.0         1
205.0         1
Name: ndays_act, Length:

In [12]:
data['nplay_video'].value_counts().sort_index() #checking if column "nplay_video" has 0 value.

1.0        16968
2.0        11000
3.0         8371
4.0         6995
5.0         5992
6.0         5373
7.0         4714
8.0         4296
9.0         4076
10.0        3620
11.0        3453
12.0        3187
13.0        2853
14.0        2641
15.0        2453
16.0        2401
17.0        2138
18.0        2085
19.0        1887
20.0        1835
21.0        1803
22.0        1671
23.0        1565
24.0        1501
25.0        1553
26.0        1412
27.0        1330
28.0        1219
29.0        1179
30.0        1205
           ...  
7847.0         1
8012.0         1
8049.0         1
8052.0         1
8160.0         1
8223.0         1
8277.0         1
8469.0         1
8632.0         1
8727.0         1
9209.0         1
9310.0         1
9409.0         1
9473.0         1
9478.0         1
9509.0         1
9652.0         1
9754.0         1
9838.0         1
10668.0        1
11458.0        1
11491.0        1
12983.0        1
14523.0        1
15487.0        1
17314.0        1
19179.0        1
24564.0       

In [13]:
data['nchapters'].value_counts().sort_index() #checking if column "nchapters" has 0 value.

1.0     121837
2.0     110085
3.0      52296
4.0      24937
5.0      13838
6.0       8536
7.0       6556
8.0       5009
9.0       4091
10.0      3598
11.0      3258
12.0      7987
13.0      2053
14.0      2021
15.0      2684
16.0      2890
17.0      1877
18.0      3411
19.0       473
20.0       624
21.0       424
22.0       150
23.0       149
24.0       157
25.0       250
26.0       339
27.0       352
28.0       193
29.0       179
30.0       181
31.0       385
32.0       657
33.0       511
34.0       271
35.0        16
36.0        23
37.0        15
38.0         8
39.0        10
40.0        12
41.0         5
42.0        11
43.0         6
44.0         3
45.0         3
46.0         3
47.0         5
48.0         6
Name: nchapters, dtype: int64

#### Considering that all 4 tested columns have no value 0 we will now assign 0 to all rows that have NaN value

In [15]:
data['nevents'].fillna(0, inplace = True)

In [18]:
data['ndays_act'].fillna(0, inplace = True)

In [19]:
data['nplay_video'].fillna(0, inplace = True)

In [20]:
data['nchapters'].fillna(0, inplace = True)

In [23]:
data.head(20)

,course_id,userid_DI,viewed,explored,certified,final_cc_cname_DI,LoE_DI,YoB,gender,grade,start_time_DI,last_event_DI,nevents,ndays_act,nplay_video,nchapters,nforum_posts
0,HarvardX/CB22x/2013_Spring,MHxPC130442623,0,0,0,United States,NaN,NaN,NaN,0,2012-12-19,2013-11-17,0.0,9.0,0.0,0.0,0
1,HarvardX/CS50x/2012,MHxPC130442623,1,0,0,United States,NaN,NaN,NaN,0,2012-10-15,NaN,0.0,9.0,0.0,1.0,0
2,HarvardX/CB22x/2013_Spring,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2013-02-08,2013-11-17,0.0,16.0,0.0,0.0,0
3,HarvardX/CS50x/2012,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2012-09-17,NaN,0.0,16.0,0.0,0.0,0
4,HarvardX/ER22x/2013_Spring,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2012-12-19,NaN,0.0,16.0,0.0,0.0,0
5,HarvardX/PH207x/2012_Fall,MHxPC130275857,1,1,0,United States,NaN,NaN,NaN,0,2012-09-17,2013-05-23,502.0,16.0,50.0,12.0,0
6,HarvardX/PH278x/2013_Spring,MHxPC130275857,0,0,0,United States,NaN,NaN,NaN,0,2013-02-08,NaN,0.0,16.0,0.0,0.0,0
7,HarvardX/CB22x/2013_Spring,MHxPC130539455,1,0,0,France,NaN,NaN,NaN,0,2013-01-01,2013-05-14,42.0,6.0,0.0,3.0,0
8,HarvardX/CB22x/2013_Spring,MHxPC130088379,1,0,0,United States,NaN,NaN,NaN,0,2013-02-18,2013-03-17,70.0,3.0,0.0,3.0,0
9,HarvardX/CS50x/2012,MHxPC130088379,1,0,0,United States,NaN,NaN,NaN,0,2012-10-20,NaN,0.0,12.0,0.0,3.0,0


In [ ]:
# data.dropna(axis=0, how='any', inplace=True)  ACTIVATE this line only when all data is cleaned and ready